In [27]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score,classification_report
from xgboost import XGBClassifier
import optuna
import matplotlib.pyplot as plt

In [29]:
data=pd.read_csv('C:\\Users\\owner\\OneDrive\\Desktop\\train (2).csv')

In [30]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [31]:
data.drop('Cabin',axis=1, inplace=True)
data['Age'].fillna(data['Age'].median(),inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0],inplace=True)

C:\Users\owner\AppData\Local\Temp\ipykernel_19860\2856279617.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].median(),inplace=True)
C:\Users\owner\AppData\Local\Temp\ipykernel_19860\2856279617.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

In [32]:
data = pd.get_dummies(data,columns=['Sex','Embarked'],drop_first=True)

In [33]:
data.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)

In [34]:
X=data.drop(['Survived'],axis=1)
y=data['Survived']

In [35]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [43]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate',0.01,0.3),
        'max_depth': trial.suggest_int('max_depth',3,10),
        'n_estimators': trial.suggest_int('n_estimators',50,1000),
        'use_label_encoder':False,
        'eval_metric':'logloss'
    }
    model = XGBClassifier(**param)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    return accuracy_score(y_test,y_pred)

In [44]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=40)
print(study.best_value)

[I 2025-05-09 10:32:26,530] A new study created in memory with name: no-name-46e93051-3d8c-4ab1-a113-d3ccafbe858c
C:\Users\owner\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [10:32:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2025-05-09 10:32:26,566] Trial 0 finished with value: 0.8044692737430168 and parameters: {'learning_rate': 0.018988746779833254, 'max_depth': 3, 'n_estimators': 66}. Best is trial 0 with value: 0.8044692737430168.
C:\Users\owner\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [10:32:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWa

0.8491620111731844


In [45]:
print(study.best_params)

{'learning_rate': 0.08848017807834105, 'max_depth': 7, 'n_estimators': 54}


In [46]:
import xgboost
print(xgboost.__version__)

2.1.1


In [51]:
best_model=XGBClassifier(**study.best_params,use_label_encoder=False,eval_metric='logloss')
best_model.fit(X_train,y_train)
y_pred=best_model.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
print("Accuracy: ",accuracy)


Accuracy:  0.8491620111731844


C:\Users\owner\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [10:32:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
